In [1]:
import pandas as pd

sedes = pd.read_csv('./sedes_2.csv')
sedes['Direcci\xc3\xb3n'] = sedes['Direcci\xc3\xb3n'].str[12:]
sedes['Nombre'] = sedes['Universidad'] +', '+sedes['Facultad'] 

oferta = pd.read_csv('./oferta.csv')

In [8]:
ambito = pd.read_csv('./oferta_ambito.csv')
provincia = pd.read_csv('./oferta_provincia.csv')
disciplina = pd.read_csv('./oferta_disciplina.csv')

In [136]:
provincia['Provincia'].unique()

array(['Buenos Aires', 'Ciudad Autonoma de Buenos Aires',
       'C\xc3\xb3rodoba', 'Catamarca', 'Chaco', 'Chubut', 'Corrientes',
       'Entre R\xc3\xados', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja',
       'Mendoza', 'Misiones', 'Neuqu\xc3\xa9n', 'Rio Negro', 'Salta',
       'San Juan', 'San Luis', 'Santa Cruz', 'Santa F\xc3\xa9',
       'Santiago del Estero', 'Tierra del Fuego', 'Tucum\xc3\xa1n'], dtype=object)

In [13]:
disciplina.merge(provincia).merge(ambito, how = 'left').to_csv('./oferta_apd.csv')

In [126]:
sedes = pd.read_csv('./sedes_geoloc_.csv', index_col=0)
oferta = pd.read_csv('./oferta_apd.csv')

estrategicas = pd.read_csv('./estrategicas.csv')

# Marcar carreras estrategicas

import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout
import re

# estrategicas['keyword'] = 
# estrategicas['T\xc3\xadtulo'].str.replace('Analista', "")
pd.options.display.max_rows = 999

estrategicas['T\xc3\xadtulo'] = estrategicas['T\xc3\xadtulo'].str.replace(r"\(.*\)","")
pattern = '|'.join(['Analista ', 'Ciencias Básicas ', 'E ', 'El ','Los ', '– ', ', ','De ', 'En ', 'La ', 'Para ', 'Y ',
                    '/a', '/o', 'Licenciatura ', 'Tecnicatura ', 'Universitaria ','Con ', 'Orientada ','Universitario',
                    'Bachiller ', 'Especialista ', 'Especializado ', u'Técnica', u'Técnico', 'Educación', 'Ciclo', 'Inicial', 
                   'con', 'nivel', 'orientación', 'Alternativa', u'Básica', u'Básicas', u'Básico', 'Licenciado',
                   'Profesional', 'Profesor', 'Profesor/a', 'Profesorado', 'Nivel', 'No', 'Para', 'Grado', 'Medio', 'Medios',
                   'Tercer', 'Terciario', 'Recursos', 'Artes', 'Contra'])

estrategicas['T\xc3\xadtulo'] = estrategicas['T\xc3\xadtulo'].str.replace(pattern, '').str.strip().drop_duplicates()
estrategicas_kwd_list = sorted(list(set(' '.join(estrategicas['T\xc3\xadtulo'].dropna().values).split(" ")) - {
            '', 'Básicas', 'Básica', 'Básico', 'Gestión', 'Técnico', 'Educación', 'orientación','Técnica','Orientación', 'Mención', 'ado', 's', 'Del', 'A', 'Las', 'Por'}))

oferta['Estrategica'] = oferta['T\xc3\xadtulo'].str.contains('|'.join(estrategicas_kwd_list))

oferta.to_csv('./oferta_apde.csv', index = False)

In [124]:
estrategicas_kwd_list

['Accionamientos',
 'Acuicultura',
 'Acu\xc3\xadcola',
 'Administraci\xc3\xb3n',
 'Administrativa',
 'Aeroespaciales',
 'Aeronaval',
 'Aeronaves',
 'Aeron\xc3\xa1utico',
 'Agraria',
 'Agrarias',
 'Agrario',
 'Agrimensor',
 'Agro-industrial',
 'Agroalimentaria',
 'Agroalimentario',
 'Agroalimentarios',
 'Agroalimentos',
 'Agrobiotecnolog\xc3\xada',
 'Agroecolog\xc3\xada',
 'Agroindustria',
 'Agroindustriales',
 'Agroindustrias',
 'Agromec\xc3\xa1nica',
 'Agronegocios',
 'Agropecuaria',
 'Agropecuarias',
 'Agropecuario',
 'Agropecuarios',
 'Agr\xc3\xadcola',
 'Agr\xc3\xadcolas',
 'Agr\xc3\xb3nomo',
 'Aguas',
 'Alimentaci\xc3\xb3n',
 'Alimentaria',
 'Alimentarias',
 'Alimenticia',
 'Alimentos',
 'Ambiental',
 'Ambientales',
 'Ambiente',
 'Analista',
 'Anal\xc3\xadtica',
 'Animal',
 'An\xc3\xa1lisis',
 'Apicultura',
 'Aplicaciones',
 'Aplicada',
 'Aprovechamiento',
 'Ap\xc3\xadcola',
 'Ap\xc3\xadcolas',
 'Arboricultura',
 'Areas',
 'Armamentos',
 'Armas',
 'Arom\xc3\xa1ticas',
 'Arquitecto

In [17]:
import json
import time
import urllib
import urllib2

def geolocate_rest_api(searchtext, APP_ID = None, APP_CODE = None, BACKOFF_TIME = 30):

    base_url = 'https://geocoder.api.here.com/6.2/geocode.json'

    # This joins the parts of the URL together into one string.
    url = base_url + '?' + urllib.urlencode({
        'searchtext': "%s" % (searchtext),
        'app_id': APP_ID,
        'app_code': APP_CODE,
    })
#     print url
    response = str(urllib2.urlopen(url).read())
    result = json.loads(response.replace('\\n', ''))
    return result


def set_geolocation_values(df):
    if result['Response']['View'] != []:
        core_result = result['Response']['View'][0]['Result'][0]['Location']
        pos = core_result['NavigationPosition']
        address = core_result['Address']

        if 'Country' in address.keys(): df.set_value(i,'Pais', address[u'Country'])
        if 'State' in address.keys(): df.set_value(i,'Provincia', address[u'State'])
        if 'County' in address.keys(): df.set_value(i,'Municipio', address[u'County'])
#         if 'Label' in address.keys(): df.set_value(i,'Direccion', address[u'Label'])
    #     df.set_value(i,'location_id', address[u'LocationId'])
        if 'Longitude' in pos[0].keys(): df.set_value(i,'lng', pos[0]['Longitude'])
        if 'Latitude' in pos[0].keys(): df.set_value(i,'lat', pos[0]['Latitude'])
    else:
        pass

In [25]:
sedes = oferta_apde[['Universidad', 'Facultad', 'Domicilio', 'Provincia']].drop_duplicates()

In [29]:
oferta_apde = pd.read_csv('./oferta_apde.csv')
dataframe = sedes[['Domicilio']].drop_duplicates()

col = 'Domicilio'
APP_ID = 'YpwwDaCIpYuxqHCMBAZY'
APP_CODE = 'aI2CPGUAgGi_DwMsTXQK2Q'

for i, row in dataframe.iterrows():
    result = geolocate_rest_api(row[col], APP_ID, APP_CODE)
#     print row[col]
    
    if result:
        set_geolocation_values(dataframe)

    if i%10 == 0:
        print 'processed row '+str(i)

processed row 0
processed row 30
processed row 50
processed row 60
processed row 80
processed row 100
processed row 110
processed row 120
processed row 140
processed row 150
processed row 170
processed row 200
processed row 230
processed row 260
processed row 280
processed row 300
processed row 310
processed row 350
processed row 360
processed row 370
processed row 380
processed row 390
processed row 420
processed row 450
processed row 510
processed row 530
processed row 580
processed row 660
processed row 680
processed row 690
processed row 900
processed row 940
processed row 1080
processed row 1310
processed row 1330
processed row 1360
processed row 1390
processed row 1400
processed row 1420
processed row 1430
processed row 1460
processed row 1470
processed row 1490
processed row 1510
processed row 1540
processed row 1560
processed row 1600
processed row 1610
processed row 1800
processed row 1810
processed row 1900
processed row 1940
processed row 2040
processed row 2140
processed ro

In [62]:
sedes.loc[sedes.Provincia == 'Córodoba', 'Provincia'] = sedes.loc[sedes.Provincia == 'Córodoba','Provincia'].str.replace('Córodoba','Córdoba')

In [63]:
sedes_loc = sedes.merge(dataframe, on = 'Domicilio',how = 'left').sort_values(by = 'Domicilio').fillna('')

import unidecode

for col in ['Provincia_x', 'Provincia_y']:
    sedes_loc[col] = [unidecode.unidecode(acc).title() for acc in sedes_loc[col].str.decode('utf8')]

In [72]:
sedes_loc_clean = sedes_loc.loc[sedes_loc.Provincia_x == sedes_loc.Provincia_y]
sedes_loc_clean = sedes_loc_clean.drop(['Provincia_x', 'Provincia_y'], axis = 1)

sedes_loc_clean.to_csv('sedes_loc_clean.csv')

### Drafts

In [76]:
AMBA_lng = (sedes_loc_clean['lng'] > -59.2) & (sedes_loc_clean['lng'] <= -57.8)
AMBA_lat = (sedes_loc_clean['lat'] < -34.05) & (sedes_loc_clean['lat'] >= -35.1)

sedes_AMBA = sedes_loc_clean.loc[AMBA_lng & AMBA_lat]

In [127]:
oferta_apde = pd.read_csv('./oferta_apde.csv')
oferta_estrat_AMBA = oferta_apde.loc[(oferta_apde['Ámbito'] == 'Público') & oferta_apde['Estrategica']].merge(sedes_AMBA, on = ['Universidad', 'Facultad', 'Domicilio'], how = 'left').dropna()

In [137]:
oferta_apde

,Unnamed: 0,Universidad,Facultad,Título,Tipo de Título,Duración,Condiciones de Ingreso,Domicilio,Teléfono,Web,Mail,Disciplina,Provincia,Ámbito,Estrategica
0,0,"Instituto Universitario del Ejército ""Mayor Francisco Romero""",Unidad Académica Escuela Superior Técnica,Técnico en Geomática,Otros Pregrados,3 Años,Exámen de Ingreso,Av Cabildo 15 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 47793300,www.ingenieriaest.iese.edu.ar,esting@iese.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Público,True
1,1,"Instituto Universitario del Ejército ""Mayor Francisco Romero""",Unidad Académica Escuela Superior Técnica,Técnico en Geomática,Otros Pregrados,3 Años,Exámen de Ingreso,Av Cabildo 15 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 47793300,www.ingenieriaest.iese.edu.ar,esting@iese.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Público,True
2,2,Universidad Abierta Interamericana,Facultad de Turismo y Hospitalidad,Guía Universitario en Turismo,Otros Pregrados,3 Años,Asistencia y Aprobación a Curso de Ingreso o Nivelación,Chacabuco 90 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 43427788,www.uai.edu.ar,contacto@uai.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Privado,False
3,3,Universidad Abierta Interamericana,Facultad de Turismo y Hospitalidad,Guía Universitario en Turismo,Otros Pregrados,3 Años,Asistencia y Aprobación a Curso de Ingreso o Nivelación,Chacabuco 90 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 43427788,www.uai.edu.ar,contacto@uai.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Privado,False
4,4,Universidad Abierta Interamericana,Facultad de Turismo y Hospitalidad,Licenciado en Hotelería,Grado,4 Años,Asistencia y Aprobación a Curso de Ingreso o Nivelación,Chacabuco 90 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 43427788,www.uai.edu.ar,contacto@uai.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Privado,False
5,5,Universidad Abierta Interamericana,Facultad de Turismo y Hospitalidad,Licenciado en Hotelería,Grado,4 Años,Asistencia y Aprobación a Curso de Ingreso o Nivelación,Chacabuco 90 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 43427788,www.uai.edu.ar,contacto@uai.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Privado,False
6,6,Universidad Abierta Interamericana,Facultad de Turismo y Hospitalidad,Licenciado en Turismo,Grado,4 Años,Asistencia y Aprobación a Curso de Ingreso o Nivelación,Chacabuco 90 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 43427788,www.uai.edu.ar,contacto@uai.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Privado,False
7,7,Universidad Abierta Interamericana,Facultad de Turismo y Hospitalidad,Licenciado en Turismo,Grado,4 Años,Asistencia y Aprobación a Curso de Ingreso o Nivelación,Chacabuco 90 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 43427788,www.uai.edu.ar,contacto@uai.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Privado,False
8,8,Universidad Abierta Interamericana,Facultad de Turismo y Hospitalidad,Técnico Universitario en Hotelería,Título Intermedio,3 Años,Asistencia y Aprobación a Curso de Ingreso o Nivelación,Chacabuco 90 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 43427788,www.uai.edu.ar,contacto@uai.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Privado,False
9,9,Universidad Abierta Interamericana,Facultad de Turismo y Hospitalidad,Técnico Universitario en Hotelería,Título Intermedio,3 Años,Asistencia y Aprobación a Curso de Ingreso o Nivelación,Chacabuco 90 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires,(0011) 43427788,www.uai.edu.ar,contacto@uai.edu.ar,Demografía y Geografía,Ciudad Autonoma de Buenos Aires,Privado,False


In [135]:
pd.set_option('display.max_colwidth', -1)
oferta_estrat_AMBA.groupby('Disciplina')['Título'].unique()

Disciplina
Arqueología                                                    [Licenciado en Paleontología]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 